## Students: Yuying Zhou, Xirong Cao

In [1]:
%load_ext sql

### connect to database lab3.db

In [2]:
%sql sqlite:///lab3.db

'Connected: @lab3.db'

### check tables' existence 

In [3]:
%%sql
select * from student

 * sqlite:///lab3.db
Done.


SId,Sname,Sage,Sgender,GPA
01,Peter,1990-01-01,M,3.7
02,Sam,1990-12-21,M,2.0
03,Lee,1990-05-20,M,3.0
04,June,1990-08-06,M,0.0
05,Eda,1991-12-01,F,3.0
06,Eloise,1992-03-01,F,0.0
07,Silas,1989-07-01,F,4.0
09,Hazel,2001-12-20,F,None
10,Oscar,2002-12-25,F,None
11,Oscar,2002-12-30,F,None


In [4]:
%%sql
select * from instructor

 * sqlite:///lab3.db
Done.


PId,Pname
01,Everett
02,Miles
03,June


In [5]:
%%sql
select * from course

 * sqlite:///lab3.db
Done.


CId,Cname,PId
01,History,02
02,Calculus,01
03,CS-I,03


In [6]:
%%sql
select * from grade

 * sqlite:///lab3.db
Done.


SId,CId,score
01,01,80
01,02,90
01,03,99
02,01,70
02,02,60
02,03,80
03,01,80
03,02,80
03,03,80
04,01,50


### Question 1: Find the student id that has all the grades in between 80 and 90

#### not every student has the same number of grade

In [7]:
%%sql
select sid, count(cid) as total_grade from grade
group by sid

 * sqlite:///lab3.db
Done.


SId,total_grade
01,3
02,3
03,3
04,3
05,2
06,2
07,2


#### select those with grade between 80 and 90

In [8]:
%%sql
select sid, count(cid) as valid_grade from grade
where score between 80 and 90
group by sid

 * sqlite:///lab3.db
Done.


SId,valid_grade
01,2
02,1
03,3
05,1
07,1


#### select sid

In [9]:
%%sql
select sid from (select sid, count(cid) as valid_grade from grade
where score between 80 and 90
group by sid)
where valid_grade = (select total_grade from(select sid, count(cid) as total_grade from grade group by sid))

 * sqlite:///lab3.db
Done.


sid
03


### Question 2: Find the students, who earned a higher grade in Calculus than his/her History class

In [10]:
%%sql
select * from (
    select h.sid, s.sname, history, calculus from 
    (select sid, score as History from grade
    where cid = "01") h,(select sid, score as Calculus from grade
    where cid = "02") c, student s
    where h.sid = c.sid
    and s.sid = h.sid)
where history < calculus

 * sqlite:///lab3.db
Done.


sid,sname,history,calculus
01,Peter,80,90
05,Eda,76,87


### Question 3: Find the students, who earned a more than 70 average grade across all the courses he/she has taken

In [11]:
%%sql
select s.sname, avg(score) as average_score
from student s, grade g
where s.sid = g.sid
group by g.sid
having average_score > 70

 * sqlite:///lab3.db
Done.


Sname,average_score
Peter,89.66666666666667
Lee,80.0
Eda,81.5
Silas,93.5


### Question 4: Find the students that did not take any courses taught by Professor Everett

In [12]:
%%sql
select distinct s.sname
from student s, grade g
where s.sid = g.sid
except 
select s.sname
from student s, grade g
where s.sid = g.sid 
and g.cid="02"

 * sqlite:///lab3.db
Done.


Sname
Eloise


### Question 5: Find the students, who took the course Calculus and have earned a grade, at least, higher than all students who took History course. Return the results in a descending order of grade

In [13]:
%%sql
select s.sname, g.score as Calculus_Score from grade g, student s
where cid="02"
and score > (select max(score) as highest_history_score from grade
where cid="01")
and s.sid = g.sid
order by sname DESC

 * sqlite:///lab3.db
Done.


Sname,Calculus_Score
Silas,89
Peter,90
Eda,87


### Question 6: Find the students, who did not take all the courses as presented in the Course table

In [14]:
%%sql
select sid, count(cid) as total_course from grade
group by sid

 * sqlite:///lab3.db
Done.


SId,total_course
01,3
02,3
03,3
04,3
05,2
06,2
07,2


In [15]:
%%sql
select g.sid, s.sname, g.total_course from (
    select sid, count(cid) as total_course from grade 
    group by sid) as g, student s
where total_course < (select count(*) from course)
and g.sid = s.sid

 * sqlite:///lab3.db
Done.


sid,Sname,total_course
05,Eda,2
06,Eloise,2
07,Silas,2


### Question 7: Calculate the average grades from Male and Female students

#### female average

In [16]:
%%sql
select avg(score) as female_average 
from student s, grade g
where s.sid=g.sid
and s.sgender = "F"

 * sqlite:///lab3.db
Done.


female_average
69.16666666666667


#### male average

In [17]:
%%sql
select avg(score) as male_average
from student s, grade g
where s.sid=g.sid
and s.sgender = "M"

 * sqlite:///lab3.db
Done.


male_average
68.25


### Question 8: Find the courses with highest enrollment

In [18]:
%%sql
select c.cname, count(g.cid) as enrollment
from course c, grade g
where c.cid=g.cid
group by g.cid

 * sqlite:///lab3.db
Done.


Cname,enrollment
History,6
Calculus,6
CS-I,6


### Question 9: For each course, find the students who earned more than its average grade

In [19]:
%%sql 
select g.cid, avg(g.score) as course_average
from grade g
group by g.cid

 * sqlite:///lab3.db
Done.


CId,course_average
01,64.5
02,72.66666666666667
03,68.5


In [20]:
%%sql
select g.cid, s.sname, score, course_average from grade g, (
    select g.cid, avg(g.score) as course_average
    from grade g
    group by g.cid) as average, student s
where average.cid = g.cid
and score > course_average
and g.sid = s.sid
order by g.cid

 * sqlite:///lab3.db
Done.


CId,Sname,score,course_average
01,Peter,80,64.5
01,Sam,70,64.5
01,Lee,80,64.5
01,Eda,76,64.5
02,Peter,90,72.66666666666667
02,Lee,80,72.66666666666667
02,Eda,87,72.66666666666667
02,Silas,89,72.66666666666667
03,Peter,99,68.5
03,Sam,80,68.5


### Question 10: Add column "GPA" in the student table and calculate the GPA for each student. <br>
#### Based on the grade, >=90 is A(4.0), >=87 is A-(3.7), >=83 is B+ (3.3), >=80 is B(3.0), >=77 is B-(2.7), >=70 is C (2.0), <=70 and >60 is D(1.0) and <=60 is F(0)

#### 1) drop GPA column if exist

In [21]:
%%sql
alter Table student
drop column GPA

 * sqlite:///lab3.db
Done.


[]

#### 2) add new GPA columns

In [22]:
%%sql
alter Table student 
add GPA float

 * sqlite:///lab3.db
Done.


[]

#### 3) calculating average for each student 

In [23]:
%%sql
select s.sid, s.sname, g.average
from (select sid, avg(score) as average
        from grade
        group by sid) as g, student s
where s.sid = g.sid

 * sqlite:///lab3.db
Done.


SId,Sname,average
01,Peter,89.66666666666667
02,Sam,70.0
03,Lee,80.0
04,June,33.333333333333336
05,Eda,81.5
06,Eloise,32.5
07,Silas,93.5


#### 4) updating student table other table g

In [24]:
%%sql
update student 
set 
GPA = g.average
from (select s.sid, s.sname, g.average
from (select sid, avg(score) as average
        from grade
        group by sid) as g, student s
where s.sid = g.sid) g
where g.sid = student.sid

 * sqlite:///lab3.db
7 rows affected.


[]

#### 5) display new table

In [25]:
%%sql 
select * from student

 * sqlite:///lab3.db
Done.


SId,Sname,Sage,Sgender,GPA
01,Peter,1990-01-01,M,89.66666666666667
02,Sam,1990-12-21,M,70.0
03,Lee,1990-05-20,M,80.0
04,June,1990-08-06,M,33.333333333333336
05,Eda,1991-12-01,F,81.5
06,Eloise,1992-03-01,F,32.5
07,Silas,1989-07-01,F,93.5
09,Hazel,2001-12-20,F,None
10,Oscar,2002-12-25,F,None
11,Oscar,2002-12-30,F,None


#### 6) update new student table attribute value by replacing original value with new one

In [26]:
%%sql
update student
set GPA = CASE
when GPA >= 90 then 4.0
when GPA >= 87 then 3.7
when GPA >= 83 then 3.3
when GPA >= 80 then 3.0
when GPA >= 77 then 2.7
when GPA >= 70 then 2.0
when GPA <= 70 and GPA >60 then 1.0
when GPA <= 60 then 0
end

 * sqlite:///lab3.db
12 rows affected.


[]

In [27]:
%%sql
select * from student

 * sqlite:///lab3.db
Done.


SId,Sname,Sage,Sgender,GPA
01,Peter,1990-01-01,M,3.7
02,Sam,1990-12-21,M,2.0
03,Lee,1990-05-20,M,3.0
04,June,1990-08-06,M,0.0
05,Eda,1991-12-01,F,3.0
06,Eloise,1992-03-01,F,0.0
07,Silas,1989-07-01,F,4.0
09,Hazel,2001-12-20,F,None
10,Oscar,2002-12-25,F,None
11,Oscar,2002-12-30,F,None
